<a href="https://colab.research.google.com/github/Diangelion/research-visualizer/blob/main/research_visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-community langchain-huggingface PyPDF2

# Image Generator

In [ ]:
import io
import os
import torch
import matplotlib.pyplot as plt

from PIL import Image
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFacePipeline
from diffusers import StableDiffusion3Pipeline
from PyPDF2 import PdfReader
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM
)
from diffusers import StableDiffusionPipeline

In [ ]:
#=======================
# Configuration
#=======================
CONFIG = {
    "RESET_DB": False,
    "text_gen": {
        "task": "text-generation",
        "repo_id": "Qwen/Qwen3-1.7B"
    },
    "summarize": {
        "task": "summarization",
        "repo_id": "nsi319/legal-led-base-16384"
    },
    "image_gen": {
        "task": "image-to-text",
        "repo_id": "sd-legacy/stable-diffusion-v1-5"
    }
}

In [ ]:
def get_pdf():
  uploaded = files.upload()

  if len(uploaded) != 1:
    raise ValueError("Please upload exactly ONE PDF file")

  filename, content = next(iter(uploaded.items()))

  if not filename.lower().endswith('.pdf'):
    raise ValueError("Only PDF files are accepted")

  # Read PDF directly from bytes
  pdf_reader = PdfReader(io.BytesIO(content))
  file_text = "\n\n".join([page.extract_text() for page in pdf_reader.pages])

  return file_text

In [ ]:
def create_summarizer(paper_text):
  tokenizer = AutoTokenizer.from_pretrained(CONFIG["summarize"]["repo_id"])
  model = AutoModelForSeq2SeqLM.from_pretrained(CONFIG["summarize"]["repo_id"])
  padding = "max_length"
  input_tokenized = tokenizer.encode(
      paper_text,
      return_tensors='pt',
      padding=padding,
      pad_to_max_length=True,
      max_length=16384,
      truncation=True
  )
  summary_ids = model.generate(
      input_tokenized,
      num_beams=4,
      no_repeat_ngram_size=3,
      length_penalty=2,
      min_length=500,
      max_length=1000
  )
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
  return { "summary": summary }

In [ ]:
def create_visualizer():
  tokenizer = AutoTokenizer.from_pretrained(CONFIG["text_gen"]["repo_id"])
  model = AutoModelForCausalLM.from_pretrained(
      CONFIG["text_gen"]["repo_id"],
      torch_dtype="auto",
      device_map="auto"
  )
  pipe = pipeline(
    task=CONFIG["text_gen"]["task"],
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=32768
  )
  visualizer_model = HuggingFacePipeline(pipeline=pipe)
  template = (
      """
      You are a scientific research visualizator expert.
      Create a DETAILED description for an illustration to accompany the summary of a research paper below.

      SUMMARY
      {summary}
      """
  )
  visualizer_prompt = PromptTemplate.from_template(template)
  return visualizer_model, visualizer_prompt

In [ ]:
def generate_image(image_prompt_text):
  pipe = StableDiffusionPipeline.from_pretrained(
      CONFIG["image_gen"]["repo_id"],
      torch_dtype=torch.float16
  )
  pipe = pipe.to("cuda")
  image = pipe(image_prompt_text).images[0]

  plt.imshow(image)
  plt.axis('off')
  plt.show()

  return { "ai_prompt": image_prompt_text}

In [ ]:
def get_chain(
    visualizer_model: HuggingFacePipeline,
    visualizer_prompt: PromptTemplate
):
  chain = (
    (lambda x: create_summarizer(x["text_paper"]))
    | visualizer_prompt
    | visualizer_model
    | StrOutputParser()
    | generate_image
  )

  return chain

In [ ]:
def main():
  try:
    file_text = get_pdf()
    visualizer_model, visualizer_prompt = create_visualizer()
    chain = get_chain(visualizer_model, visualizer_prompt)
    response = chain.invoke({ "text_paper": file_text })
    print(f"Response: {response}")
  except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
if __name__ == "__main__":
  main()